<a href="https://colab.research.google.com/github/dnanper/TEST-langchain/blob/main/test03_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-core>=0.3.47 langchain-openai>=0.3.9 langchain-community==0.3.16

In [2]:
!pip install langsmith

In [3]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchaintest"

Enter LangSmith API Key: ··········


In [4]:
import os
from getpass import getpass

os.environ["OPENROUTER_API_KEY"] = os.getenv("OPENROUTER_API_KEY") or getpass(
    "Enter OpenRouter API Key: "
)

openrouter_model = "deepseek/deepseek-r1:free"

Enter OpenRouter API Key: ··········


In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    temperature=0.0,
    api_key = os.getenv("OPENROUTER_API_KEY"),
    base_url = "https://openrouter.ai/api/v1",
    model=openrouter_model
    )

##Buffer Memory

### Using a built-in class: InMemoryChatMessageHistory as Message_Histoy

In [5]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, ChatPromptTemplate

In [6]:
system_prompt = "You are a helpful assistant called Zeta."

In [20]:
prompt_template_1 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [16]:
prompt_template_1.input_variables

['history', 'input']

In [21]:
pipeline_1 = prompt_template_1 | llm

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [12]:
chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
  if session_id not in chat_map:
    chat_map[session_id] = InMemoryChatMessageHistory()
  return chat_map[session_id]

In [13]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [23]:
pipeline_1_history = RunnableWithMessageHistory(
    pipeline_1,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [25]:
pipeline_1_history.invoke(
    {"query": "hi i'm An"},
    config = {"session_id": "01"}
)

AIMessage(content='Hi An! 👋 Nice to meet you. How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 364, 'prompt_tokens': 53, 'total_tokens': 417, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747969380-uqykgKbSBAEBTNn0SkyV', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--862bf3d1-8d79-41c7-bad3-496ef141b585-0', usage_metadata={'input_tokens': 53, 'output_tokens': 364, 'total_tokens': 417, 'input_token_details': {}, 'output_token_details': {}})

In [26]:
pipeline_1_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "01"}
)

AIMessage(content="Your name is **An**! 😊 Let me know if there's anything specific you'd like help with today.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 312, 'prompt_tokens': 81, 'total_tokens': 393, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747969405-AWbs3oAIsCMm60pYSMX7', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--95091b9e-aa45-4f7b-b689-c29d75069982-0', usage_metadata={'input_tokens': 81, 'output_tokens': 312, 'total_tokens': 393, 'input_token_details': {}, 'output_token_details': {}})

## BufferWindowMemory

### From now, all the data structure to store History Messages must be built

In [27]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

In [29]:
class BufferWindowMemory(BaseChatMessageHistory, BaseModel):
  messages: list[BaseMessage] = Field(default_factory=list)
  k: int = Field(default_factory=int)

  def __init__(self, k:int):
    super().__init__(k=k)
    print(f"Initializing BufferWindowMessageHistory with k={k}")
  def add_messages(self, messages: list[BaseMessage])->None:
    """
    Add messages to the history, removing any messages beyond
    the last `k` messages.
    """
    self.messages.extend(messages)
    self.messages = self.messages[-self.k:]
  def clear(self)->None:
    self.messages = []

In [30]:
chat_map_2 = {}
def get_chat_history_2(session_id: str, k: int = 4) -> BufferWindowMemory:
  print(f"get_chat_history called with session_id={session_id} and k={k}")
  if session_id not in chat_map_2:
    chat_map_2[session_id] = BufferWindowMemory(k=k)
  return chat_map_2[session_id]

In [31]:
from langchain_core.runnables import ConfigurableFieldSpec

In [32]:
pipeline_2_history = RunnableWithMessageHistory(
    pipeline_1,
    get_session_history=get_chat_history_2,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
      ConfigurableFieldSpec(
        id="session_id",
        annotation=str,
        name="Session ID",
        description="The session ID to use for the chat history",
        default="id_default",
      ),
      ConfigurableFieldSpec(
        id="k",
        annotation=int,
        name="k",
        description="The number of messages to keep in the history",
        default=4,
     )
    ]
)

In [33]:
pipeline_2_history.invoke(
    {"query": "Hi, my name is An"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4
Initializing BufferWindowMessageHistory with k=4


AIMessage(content='Hello An! It looks like your message got cut off. Could you please share your full name so I can address you properly? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 21, 'total_tokens': 164, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747970173-7YgXkGVpDW7JMq44yovq', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--e503d647-f09d-4d13-ae09-959f55e892c6-0', usage_metadata={'input_tokens': 21, 'output_tokens': 143, 'total_tokens': 164, 'input_token_details': {}, 'output_token_details': {}})

In [36]:
chat_map_2["id_k4"].clear()  # clear the history

# manually insert history
chat_map_2["id_k4"].add_user_message("Hi, my name is James")
chat_map_2["id_k4"].add_ai_message("I'm an AI model called Zeta.")
chat_map_2["id_k4"].add_user_message("I'm researching the different types of conversational memory.")
chat_map_2["id_k4"].add_ai_message("That's interesting, what are some examples?")
chat_map_2["id_k4"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map_2["id_k4"].add_ai_message("That's interesting, what's the difference?")
chat_map_2["id_k4"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map_2["id_k4"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map_2["id_k4"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map_2["id_k4"].add_ai_message("Very cool!")

chat_map_2["id_k4"].messages

[HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]

In [37]:
pipeline_2_history.invoke(
    {"query": "what is my name again?"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4


AIMessage(content="I don't have access to personal information unless you explicitly share it with me. You haven't provided your name in this conversation yet! Let me know how I can assist you. 😊", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 64, 'total_tokens': 223, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747970256-Fd1Tbu4eqAMv5IQrX6ko', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--d6252bc5-38fb-4d42-9191-65640386a7f6-0', usage_metadata={'input_tokens': 64, 'output_tokens': 159, 'total_tokens': 223, 'input_token_details': {}, 'output_token_details': {}})

In [38]:
pipeline_2_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14
Initializing BufferWindowMessageHistory with k=14


AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 21, 'total_tokens': 37, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747970311-VlfNnNqLRtaHaumJMcQB', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--7a1613c4-24e7-4417-bcea-3c33e8a2b66f-0', usage_metadata={'input_tokens': 21, 'output_tokens': 16, 'total_tokens': 37, 'input_token_details': {}, 'output_token_details': {}})

In [39]:
chat_map_2["id_k14"].add_user_message("I'm researching the different types of conversational memory.")
chat_map_2["id_k14"].add_ai_message("That's interesting, what are some examples?")
chat_map_2["id_k14"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map_2["id_k14"].add_ai_message("That's interesting, what's the difference?")
chat_map_2["id_k14"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map_2["id_k14"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map_2["id_k14"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map_2["id_k14"].add_ai_message("Very cool!")

chat_map_2["id_k14"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 21, 'total_tokens': 37, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747970311-VlfNnNqLRtaHaumJMcQB', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--7a1613c4-24e7-4417-bcea-3c33e8a2b66f-0', usage_metadata={'input_tokens': 21, 'output_tokens': 16, 'total_tokens': 37, 'input_token_details': {}, 'output_token_details': {}}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I've been looking at ConversationBufferMemory and

In [40]:
pipeline_2_history.invoke(
    {"query": "what is my name again?"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14


AIMessage(content="Your name is **James**. Let me know if there's anything else you'd like to discuss! 😊", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 119, 'total_tokens': 275, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747970347-pEFJNpNYNQPSYqyll0Mu', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--47d7ee12-43a0-4365-a288-b81799821ca3-0', usage_metadata={'input_tokens': 119, 'output_tokens': 156, 'total_tokens': 275, 'input_token_details': {}, 'output_token_details': {}})

## SumaryMemory

In [53]:
from langchain_core.messages import SystemMessage

class SumaryMemory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)

    def __init__(self, llm: ChatOpenAI):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """
        Summarize the messages and add it to the history.
        """
        # self.messages.extend(messages)
        # prompt để tạo summary mới
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        # gọi llm, truyền vào summary_prompt
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=[msg.content for msg in self.messages], #summary hiện tại cho nội dung cũ
                messages=[x.content for x in messages]  #messages hiện tại, chưa được summarize
            )
        )
        # thay thế memory bằng summary mới
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [54]:
chat_map_3 = {}
def get_chat_history_3(session_id: str, llm: ChatOpenAI) -> SumaryMemory:
    if session_id not in chat_map_3:
        chat_map_3[session_id] = SumaryMemory(llm=llm)
    return chat_map_3[session_id]

In [55]:
pipeline_3_history = RunnableWithMessageHistory(
    pipeline_1,
    get_session_history=get_chat_history_3,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
    ]
)

In [56]:
pipeline_3_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content='Hi James! 👋 How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 21, 'total_tokens': 128, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747971853-DqtgsLd0bq9eg3IF1THz', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--299badfa-ee8c-464f-88f0-750574afb05d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 107, 'total_tokens': 128, 'input_token_details': {}, 'output_token_details': {}})

In [57]:
pipeline_3_history.invoke(
    {"query": "I'm researching the different types of conversational memory."},
    config={"session_id": "id_123", "llm": llm}
)
chat_map_3["id_123"].messages

[SystemMessage(content='**Updated Conversation Summary:**\n\n- **James** introduced himself and initiated the conversation.  \n- **Assistant** greeted James and offered assistance.  \n- **James** mentioned researching **types of conversational memory**, prompting the assistant to provide a detailed breakdown:  \n  - **1. Short-Term (Buffer) Memory**: Stores recent messages (e.g., last 5-10 exchanges) for basic chatbots.  \n  - **2. Summary Memory**: Condenses past interactions into concise summaries (e.g., tracking a user’s refund request).  \n  - **3. Entity-Centric Memory**: Tracks specific entities (e.g., user preferences like dietary restrictions).  \n  - **4. Long-Term (External) Memory**: Uses databases/vector stores for multi-session data (e.g., therapy bot tracking progress).  \n  - **5. Knowledge Graph Memory**: Links entities and relationships for contextual reasoning (e.g., weather in a previously visited city).  \n  - **6. Hybrid Memory**: Combines multiple approaches (e.g.

In [58]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    pipeline_3_history.invoke(
        {"query": msg},
        config={"session_id": "id_123", "llm": llm}
    )

In [59]:
chat_map_3["id_123"].messages

[SystemMessage(content='**Updated Conversation Summary:**\n\n**Buffer Window Memory (Capacity: k Messages)**  \nA fixed-size, sliding window mechanism that retains the **latest k messages**, discarding older entries.  \n\n**Key Features:**  \n1. **FIFO Behavior:**  \n   - New messages are added to the end.  \n   - When full, the oldest message is removed.  \n\n2. **Use Cases:**  \n   - Real-time analytics (e.g., tracking user actions).  \n   - Logging systems (storing recent errors/events).  \n   - Chat applications, network monitoring, or streaming data processing.  \n\n3. **Implementation:**  \n   - **Data Structures:** Queues (e.g., Python’s `deque`), circular buffers.  \n   - **Operations:**  \n     - **Add:** Append messages, remove oldest if capacity is reached.  \n     - **Retrieve:** Access all k messages in order.  \n\n4. **Edge Cases:**  \n   - **k=0:** All messages are discarded.  \n   - **k=1:** Only the latest message is stored.  \n   - **Underflow:** Fewer than k messages

In [61]:
pipeline_3_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content="**Answer:**  \nThe system does not store personal information such as names. It is designed for technical use cases like real-time data processing or logging. If you need assistance with the buffer's functionality, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 435, 'total_tokens': 529, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747972244-DpDjCyP6wGAc7eKQa6JY', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--78ecbd6d-d8ec-4ee7-b498-db79d27e8524-0', usage_metadata={'input_tokens': 435, 'output_tokens': 94, 'total_tokens': 529, 'input_token_details': {}, 'output_token_details': {}})

## SummaryBufferMemory

In [62]:
class SummaryBufferMemory(BaseChatMessageHistory, BaseModel):
  messages: list[BaseMessage]= Field(default_factory=list)
  llm: ChatOpenAI = Field(default_factory=ChatOpenAI)
  k: int = Field(default_factory=int)

  def __init__(self, llm: ChatOpenAI, k: int):
    super().__init__(llm=llm, k=k)

  def add_messages(self, messages:list[BaseMessage])->None:
    """
    Add messages to the history, removing any messages beyond
    the last `k` messages and summarizing the messages that we
    drop.
    """
    existing_summary: SystemMessage | None=None
    old_messages: list[BaseMessage] | None=None
    # Check xem ta đã có summary chưa (lưu ở index 0 của messages history)
    if len(self.messages) > 0 and isinstance(self.messages[0], SystemMessage):
      print(">> Found existing summary")
      existing_summary = self.messages.pop(0) # Pop nó ra khỏi message history->lúc này chỉ còn message BaseMessage ở trong
    # Thêm messages mới vào messages history
    self.messages.extend(messages)
    # Check xem số lượng messages có bị vượt quá k không
    if len(self.messages) > self.k:
      print(
          f">> Found {len(self.messages)} messages, dropping "
          f"oldest {len(self.messages) - self.k} messages.")
      old_messages = self.messages[:self.k] # Lấy ra những  old messages thừa, chỉ để lại k cái
      self.messages = self.messages[-self.k:]
    if old_messages is None:
      print(">> No old messages to update summary with")
      return  # nếu không có old message nào cả, tức không có gì để summarize
    # tạo prompt
    summary_prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(
            "Given the existing conversation summary and the new messages, "
            "generate a new summary of the conversation. Ensuring to maintain "
            "as much relevant information as possible."
        ),
        HumanMessagePromptTemplate.from_template(
            "Existing conversation summary:\n{existing_summary}\n\n"
            "New messages:\n{old_messages}"
        )
    ])
    # gọi llm, với prompt có existing_summary và old_mesages: tức summary mới = existing + old_mesages
    new_summary = self.llm.invoke(
        summary_prompt.format_messages(
            existing_summary=existing_summary,
            old_messages=old_messages
        )
    )
    print(f">> New summary: {new_summary.content}")
    self.messages = [SystemMessage(content=new_summary.content)] + self.messages # thêm lại summary mới vào cuối messages history

  def clear(self) -> None:
    """Clear the history."""
    self.messages = []

In [64]:
chat_map_4 = {}
def get_chat_history_4(session_id: str, llm: ChatOpenAI, k: int) -> SummaryBufferMemory:
    if session_id not in chat_map_4:
        chat_map_4[session_id] = SummaryBufferMemory(llm=llm, k=k)
    return chat_map_4[session_id]

In [65]:
pipeline_4_history = RunnableWithMessageHistory(
    pipeline_1,
    get_session_history=get_chat_history_4,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

In [68]:
pipeline_4_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123", "llm": llm, "k": 4}
)
chat_map_4["id_123"].messages
for i, msg in enumerate([
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]):
    print(f"---\nMessage {i+1}\n---\n")
    pipeline_4_history.invoke(
        {"query": msg},
        config={"session_id": "id_123", "llm": llm, "k": 4}
    )

>> No old messages to update summary with
---
Message 1
---

>> Found 6 messages, dropping oldest 2 messages.
>> New summary: **Summary of the Conversation:**

James introduced himself with the message, "Hi, my name is James." The AI responded warmly, saying, "Hi James! It's nice to meet you. How can I assist you today? 😊" This marks the start of the interaction, with the AI offering help in a friendly tone. No prior conversation history was present before this exchange.
---
Message 2
---

>> Found existing summary
>> Found 6 messages, dropping oldest 2 messages.
>> New summary: **Updated Summary of the Conversation:**  

James introduced himself with the message, "Hi, my name is James." The AI responded warmly, offering assistance. James then mentioned he was researching conversational memory types, prompting the AI to provide a detailed breakdown of seven key architectures:  

1. **Short-Term (Buffer) Memory**: Tracks recent exchanges for immediate context.  
2. **Long-Term (Persiste